In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import numpy as np
import json
from time import sleep, perf_counter as pf
import itertools as it
import datetime as dt

In [ ]:
def add_type(df, col, val, type):
    df.loc[(df[col.upper()] == val) & (df['TYPE'] == ''), 'TYPE'] = type

In [ ]:
def add_cat(df, col, val, type):
    df.loc[(df[col.upper()] == val) & (df['Category'] == ''), 'Category'] = 'Offense Against ' + type

In [ ]:
den = pd.read_csv('datasets/Denver.csv')
display(den)

In [ ]:
van = pd.read_csv('datasets/vancouver.csv')
display(van)

In [ ]:
# Drop unneeded columns
den.drop(['INCIDENT_ID', 'OFFENSE_ID', 'OFFENSE_CODE', 'OFFENSE_CODE_EXTENSION', 'GEO_X', 'GEO_Y', 'DISTRICT_ID', 'PRECINCT_ID'], 1, inplace=True)

# Add Time Info
datetime = pd.to_datetime(den['REPORTED_DATE'], format='%m/%d/%Y %I:%M:%S %p')
den['DATE'] = [d.date() for d in datetime]
den['TIME'] = [d.time() for d in datetime]

# Add Extra columns
den['City'] = 'Denver'
van['City'] = 'Vancouver'
den['TYPE'] = ''
van['Category'] = ''
den['Category'] = ''

# Add 'IS_TRAFFIC' and 'IS_FATAL'
van['IS_TRAFFIC'] = 0
van.loc[
    (van.TYPE == 'Vehicle Collision or Pedestrian Struck (with Injury)') |
    (van.TYPE == 'Vehicle Collision or Pedestrian Struck (with Fatality)'),
    'IS_TRAFFIC'
] = 1

van['IS_FATAL'] = 0
van.loc[
    (van.TYPE == 'Homicide') |
    (van.TYPE == 'Vehicle Collision or Pedestrian Struck (with Fatality)'),
    'IS_FATAL'
] = 1

den['IS_FATAL'] = 0
den.loc[
    (den.OFFENSE_CATEGORY_ID == 'murder'),
    'IS_FATAL'
] = 1

# Add Start/End time to vancouver
van['FIRST_OCCURRENCE_DATE'] = pd.NaT
van['LAST_OCCURRENCE_DATE'] = pd.NaT
van['Details'] = ''

display(den)

In [ ]:
datetime = pd.to_datetime(van[['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE']])
van.insert(1, 'DATE', [d.date() for d in datetime])
van.insert(2, 'TIME', [d.time() if not pd.isnull(d) else np.NaN for d in datetime])
van.drop(['X', 'Y', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE'], 1, inplace=True)
display(van)

In [ ]:
van.TYPE.unique()

In [ ]:
den.OFFENSE_CATEGORY_ID.unique()

In [ ]:
den.loc[den['OFFENSE_CATEGORY_ID'] == 'murder'].OFFENSE_TYPE_ID.unique()

In [ ]:
# Theft
add_type(den, 'offense_category_id', 'theft-from-motor-vehicle', 'Theft from Vehicle')
add_type(den, 'offense_category_id', 'auto-theft', 'Theft of Vehicle')
## Larceny
add_type(den, 'offense_type_id', 'theft-bicycle', 'Theft of Bicycle')
add_type(den, 'offense_category_id', 'larceny', 'Other Theft')
## Robbery
add_type(den, 'offense_type_id', 'robbery-car-jacking', 'Theft of Vehicle')
add_type(den, 'offense_category_id', 'robbery', 'Other Theft')
## Burglary
add_type(den, 'offense_type_id', 'burg-auto-theft-resd-w-force', 'Theft of Vehicle')
add_type(den, 'offense_type_id', 'burg-auto-theft-resd-no-force', 'Theft of Vehicle')
add_type(den, 'offense_type_id', 'burg-auto-theft-busn-w-force', 'Theft of Vehicle')
add_type(den, 'offense_type_id', 'burg-auto-theft-busn-no-force', 'Theft of Vehicle')
add_type(den, 'offense_category_id', 'burglary', 'Other Theft')
## White-collar-crimes
add_type(den, 'offense_type_id', 'theft-embezzle', 'Other Theft')
add_type(den, 'offense_type_id', 'theft-unauth-use-of-ftd', 'Other Theft')
add_type(den, 'offense_type_id', 'theft-fail-return-rent-veh', 'Theft of Vehicle')
add_type(den, 'offense_type_id', 'theft-of-rental-property', 'Other Theft') 

# Homocide
add_type(den, 'offense_type_id', 'traf-vehicular-homicide', 'Vehicle Collision or Pedestrian Struck (with Fatality)')
add_type(den, 'offense_category_id', 'murder', 'Homicide')
add_type(den, 'offense_category_id', 'traffic-accident', 'Vehicle Collision or Pedestrian Struck (with Injury)')

# Public Disorder 
add_type(den, 'offense_type_id', 'criminal-mischief-other', 'Mischief')
add_type(den, 'offense_type_id', 'criminal-mischief-mtr-veh', 'Mischief')
add_type(den, 'offense_type_id', 'criminal-mischief-graffiti', 'Mischief')
add_type(den, 'offense_type_id', 'riot', 'Mischief')
add_type(den, 'offense_type_id', 'loitering', 'Mischief')
add_type(den, 'offense_type_id', 'prostitution-engaging-in', 'Mischief')
add_type(den, 'offense_type_id', 'prostitution-aiding', 'Mischief')
add_type(den, 'offense_category_id', 'public-disorder', 'Offence Against a Person')

#all-other-crimes
add_type(den, 'offense_type_id', 'criminal-trespassing', 'Mischief')
add_type(den, 'offense_type_id', 'criminal-trespassing', 'Mischief')
add_type(den, 'offense_type_id', 'littering', 'Mischief') 
add_type(den, 'offense_type_id', 'public-fighting', 'Mischief')
add_type(den, 'offense_type_id', 'illegal-dumping', 'Mischief')
add_type(den, 'offense_type_id', 'eavesdropping', 'Mischief')

# aggravated
add_type(den, 'offense_category_id', 'aggravated-assault', 'Offence Against a Person')
add_type(den, 'offense_category_id', 'sexual-assault', 'Offence Against a Person')
add_type(den, 'offense_category_id', 'other-crimes-against-persons', 'Offence Against a Person')

display(den.loc[(den['TYPE'] == '') & (~den.OFFENSE_CATEGORY_ID.isin([
    'all-other-crimes',
    'drug-alcohol',
    'white-collar-crime',
    'arson',
]))])

In [ ]:
add_cat(den, 'type', 'Theft from Vehicle', 'Person')
add_cat(van, 'type', 'Theft from Vehicle', 'Person')
add_cat(den, 'type', 'Other Theft', 'Property')
add_cat(van, 'type', 'Other Theft', 'Property')
add_cat(den, 'type', 'Theft of Vehicle', 'Property')
add_cat(van, 'type', 'Theft of Vehicle', 'Property')
add_cat(den, 'type', 'Theft of Bicycle', 'Property')
add_cat(van, 'type', 'Theft of Bicycle', 'Property')
add_cat(den, 'type', 'Offence Against a Person', 'Person')
add_cat(van, 'type', 'Offence Against a Person', 'Person')
add_cat(den, 'type', 'Vehicle Collision or Pedestrian Struck (with Injury)', 'Society')
add_cat(van, 'type', 'Vehicle Collision or Pedestrian Struck (with Injury)', 'Society')
add_cat(den, 'type', 'Vehicle Collision or Pedestrian Struck (with Fatality)', 'Society')
add_cat(van, 'type', 'Vehicle Collision or Pedestrian Struck (with Fatality)', 'Society')
add_cat(den, 'type', 'Homicide', 'Person')
add_cat(van, 'type', 'Homicide', 'Person')
add_cat(den, 'type', 'Mischief', 'Society')
add_cat(van, 'type', 'Mischief', 'Society')
add_cat(den, 'type', 'Break and Enter Residential/Other', 'Property')
add_cat(van, 'type', 'Break and Enter Residential/Other', 'Property')
add_cat(den, 'type', 'Break and Enter Commercial', 'Property')
add_cat(van, 'type', 'Break and Enter Commercial', 'Property')

add_cat(den, 'offense_category_id', 'all-other-crimes', 'Society')
add_cat(den, 'offense_category_id', 'aggravated-assault', 'Person')
add_cat(den, 'offense_category_id', 'sexual-assault', 'Person')
add_cat(den, 'offense_category_id', 'white-collar-crime', 'Society')
add_cat(den, 'offense_category_id', 'arson', 'Property')
add_cat(den, 'offense_category_id', 'drug-alcohol', 'Society')

display(den.loc[den['Category'] == ''])
display(van.loc[van['Category'] == ''])

In [ ]:
display(van)

In [ ]:
den.drop(['OFFENSE_TYPE_ID', 'REPORTED_DATE', 'IS_CRIME'], axis=1, inplace=True)
display(den)

In [ ]:
van.columns=['TYPE', 'DATE', 'TIME', 'ADDRESS', 'NEIGHBORHOOD', 'LAT', 'LON', 'City', 'Category', 'IS_TRAFFIC', 'IS_FATAL', 'start_dt', 'end_dt', 'Details']
den.columns=['Details', 'start_dt', 'end_dt', 'ADDRESS', 'LON', 'LAT', 'NEIGHBORHOOD', 'IS_TRAFFIC', 'DATE', 'TIME', 'City', 'TYPE', 'Category', 'IS_FATAL']
crime = pd.concat([van, den], sort=True)
crime.end_dt=pd.to_datetime(crime.end_dt)
crime.start_dt=pd.to_datetime(crime.start_dt)
firstDay = dt.datetime.strptime('2015/01/01', '%Y/%m/%d').date()
lastDay = dt.datetime.strptime('2017/12/31', '%Y/%m/%d').date()
crime = crime.loc[(crime.DATE >= firstDay) & (crime.DATE <= lastDay)]
display(crime)

In [ ]:
crime.reset_index(drop=True, inplace=True) # If not we get non-unique keys
crime.insert(0, 'Crime_key', crime.index)
display(crime)

In [ ]:
crime.to_csv('datasets/crime_output.csv', index=False)

In [ ]:
# Get Final Crime Table
f_crime = crime.drop(['ADDRESS', 'City', 'DATE', 'IS_FATAL', 'IS_TRAFFIC', 'LAT', 'LON', 'NEIGHBORHOOD'], axis=1)
f_crime.to_csv('final/crime.csv', index=False)
display(f_crime)

In [ ]:
location = crime[['ADDRESS', 'LAT', 'LON', 'NEIGHBORHOOD']]
location = location.drop_duplicates()
location.reset_index(drop=True, inplace=True)
location.insert(0, 'Location_key', location.index)
display(location)

In [ ]:
location.to_csv('datasets/location_output.csv', index=False)